# Conception BDD Billeterie JO Paris 2024

Ce notebook contient les scripts pour la conception et le remplissage de la base de données de l'application Billetrerie JO 2024.

Les données brutes sont issues du site [data.paris2024.org](https://data.paris2024.org/pages/accueil/).

En particuler, la base de données sera peuplée à partir d'évènements fictifs conçus sur la base du fichier [Paris 2024 - Sites de compétition](https://data.paris2024.org/explore/dataset/paris-2024-sites-de-competition/information/).

In [5]:
import pandas as pd

df = pd.read_csv('paris-2024-sites-de-competition.csv', sep=';')

In [6]:
df.head()

,Code_Site,Nom_Site,category_id,Sports,start_date,end_date,adress,latitude,longitude,point_geo
0,GRP,Grand Palais,venue-olympic,"Escrime (FEN), Taekwondo (TKW)",2024-07-27,2024-08-10,NaN,"48,86616355","2,3125474","48.86616355, 2.3125474"
1,CDM,Arena Champ de Mars,venue-olympic,"Judo (JUD), Lutte (WRE)",2024-07-27,2024-08-11,NaN,"48,8531","2,30252","48.8531, 2.30252"
2,RGA,Stade Roland Garros,venue-olympic,"Tennis (TEN), Boxe (BOX)",2024-07-27,2024-08-10,NaN,"48,845968","2,253522","48.845968, 2.253522"
3,TAH,Tahiti Teahupo'o,venue-olympic,Surf (SRF),2024-07-27,2024-07-30,NaN,"-17,86693","-149,25208","-17.86693, -149.25208"
4,CDM,Arena Champ de Mars,venue-paralympic,"Para Judo (JUD), Rugby fauteuil (WRU)",2024-08-29,2024-09-07,NaN,"48,8531","2,30252","48.8531, 2.30252"


In [7]:
df.drop(['adress', 'latitude', 'longitude', 'point_geo'], axis=1, inplace=True)
df = df[df['category_id']=='venue-olympic']
df.drop(['category_id'], axis=1, inplace=True)
df.head()

,Code_Site,Nom_Site,Sports,start_date,end_date
0,GRP,Grand Palais,"Escrime (FEN), Taekwondo (TKW)",2024-07-27,2024-08-10
1,CDM,Arena Champ de Mars,"Judo (JUD), Lutte (WRE)",2024-07-27,2024-08-11
2,RGA,Stade Roland Garros,"Tennis (TEN), Boxe (BOX)",2024-07-27,2024-08-10
3,TAH,Tahiti Teahupo'o,Surf (SRF),2024-07-27,2024-07-30
8,CPL,Arena Porte de la Chapelle,"Badminton (BDM), Gymnastique rythmique (GRY)",2024-07-27,2024-08-10


## Gestion des sports

J'extrais le nom et le code des sports et je garde le reste des informations.

In [45]:
import re

new_df = pd.DataFrame(columns=['Nom_Sport', 'Code_Sport', 'Nom_Site', 'Code_Site', 'start_date', 'end_date'])

idx1 = 0
for idx in df.index:
    sports = df.loc[idx, 'Sports'].split(', ')
    for sport in sports:
        sport_code = re.search(r'\((.*?)\)', sport).group(1)
        sport_name = sport[:-6]
        new_df.loc[idx1, 'Nom_Sport'] = sport_name
        new_df.loc[idx1, 'Code_Sport'] = sport_code
        new_df.loc[idx1, 'Nom_Site'] = df.loc[idx, 'Nom_Site']
        new_df.loc[idx1, 'Code_Site'] = df.loc[idx, 'Code_Site']
        new_df.loc[idx1, 'start_date'] = df.loc[idx, 'start_date']
        new_df.loc[idx1, 'end_date'] = df.loc[idx, 'end_date']
        idx1 += 1


In [47]:
new_df.head(10)

,Nom_Sport,Code_Sport,Nom_Site,Code_Site,start_date,end_date
0,Escrime,FEN,Grand Palais,GRP,2024-07-27,2024-08-10
1,Taekwondo,TKW,Grand Palais,GRP,2024-07-27,2024-08-10
2,Judo,JUD,Arena Champ de Mars,CDM,2024-07-27,2024-08-11
3,Lutte,WRE,Arena Champ de Mars,CDM,2024-07-27,2024-08-11
4,Tennis,TEN,Stade Roland Garros,RGA,2024-07-27,2024-08-10
5,Boxe,BOX,Stade Roland Garros,RGA,2024-07-27,2024-08-10
6,Surf,SRF,Tahiti Teahupo'o,TAH,2024-07-27,2024-07-30
7,Badminton,BDM,Arena Porte de la Chapelle,CPL,2024-07-27,2024-08-10
8,Gymnastique rythmique,GRY,Arena Porte de la Chapelle,CPL,2024-07-27,2024-08-10
9,Basketball,BKB,Arena Bercy,BCY,2024-07-27,2024-08-11


## Gestion des dates

Je crée un dataset avec toutes les dates quotidiennes entre start_date et end_date inclus.

In [86]:
import datetime

final_df = pd.DataFrame(columns=['Nom_Sport', 'Code_Sport', 'Nom_Site', 'Code_Site', 'Date'])
idx0 = 0
for idx in new_df.index:
    start_date = datetime.datetime.strptime(new_df.loc[idx, 'start_date'].split(' ')[0], '%Y-%m-%d')
    end_date = datetime.datetime.strptime(new_df.loc[idx, 'end_date'].split(' ')[0], '%Y-%m-%d')
    delta = end_date.date() - start_date.date()
    for i in range(delta.days + 1):
        final_df.loc[idx0, 'Nom_Sport'] = new_df.loc[idx, 'Nom_Sport']
        final_df.loc[idx0, 'Code_Sport'] = new_df.loc[idx, 'Code_Sport']
        final_df.loc[idx0, 'Nom_Site'] = new_df.loc[idx, 'Nom_Site']
        final_df.loc[idx0, 'Code_Site'] = new_df.loc[idx, 'Code_Site']
        final_df.loc[idx0, 'Date'] = (start_date + datetime.timedelta(days=i)).date()
        idx0 += 1

In [88]:
final_df.sample(10)

,Nom_Sport,Code_Sport,Nom_Site,Code_Site,Date
621,Cyclisme sur route - Contre-la-montre-individuel,CRD,Invalides,INV,2024-08-08
76,Tennis,TEN,Stade Roland Garros,RGA,2024-08-10
177,Football,FBL,Parc des Princes,PDP,2024-07-27
602,Athlétisme marathon,ATH,Invalides,INV,2024-08-05
722,Cyclisme sur route - Contre-la-montre-individuel,CRD,PontAlexandre III,ALX,2024-08-03
56,Lutte,WRE,Arena Champ de Mars,CDM,2024-08-06
123,Gymnastique rythmique,GRY,Arena Porte de la Chapelle,CPL,2024-08-08
703,Breaking,BKG,La Concorde 1,LC1,2024-08-05
18,Taekwondo,TKW,Grand Palais,GRP,2024-07-30
118,Gymnastique rythmique,GRY,Arena Porte de la Chapelle,CPL,2024-08-03


In [90]:
final_df.to_csv('paris-2024-evenements-sportifs-final.csv', index=False)